In [1]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

In [3]:
pip install tensorflow==2.12.0rc0 tensorflow-io==0.29.0 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy pandas scikit-learn tensorflow keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
np.random.seed(0)
# from sklearn.metrics import precision_score,recall_score,f1_score
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.metrics import confusion_matrix


In [7]:
data = pd.read_csv("Project1\labelled_testing_data.csv")
print("Total missing values : ",sum(list(data.isna().sum())))
data



Total missing values :  0


,timestamp,processId,threadId,parentProcessId,userId,mountNamespace,processName,hostName,eventId,eventName,stackAddresses,argsNum,returnValue,args,sus,evil
0,129.050634,382,382,1,101,4026532232,systemd-resolve,ip-10-100-1-217,41,socket,"[140159195621643, 140159192455417, 94656731598...",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0,0
1,129.051238,379,379,1,100,4026532231,systemd-network,ip-10-100-1-217,41,socket,"[139853228042507, 93935071185801, 93935080775184]",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0,0
2,129.051434,1,1,0,0,4026531840,systemd,ip-10-100-1-217,1005,security_file_open,"[140362867191588, 8103505641674583858]",4,0,"[{'name': 'pathname', 'type': 'const char*', '...",0,0
3,129.051481,1,1,0,0,4026531840,systemd,ip-10-100-1-217,257,openat,[],4,17,"[{'name': 'dirfd', 'type': 'int', 'value': -10...",0,0
4,129.051522,1,1,0,0,4026531840,systemd,ip-10-100-1-217,5,fstat,[140362867189385],2,0,"[{'name': 'fd', 'type': 'int', 'value': 17}, {...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188962,496.629003,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188963,496.629025,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188964,496.629047,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1
188965,496.629069,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"[{'name': 'sockfd', 'type': 'int', 'value': 42...",1,1


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout


In [9]:
non_numeric_columns = data.select_dtypes(include=['object']).columns
le = LabelEncoder()
for column in non_numeric_columns:
    data[column] = le.fit_transform(data[column])

In [10]:
# Assume the last column is the label
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

    
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for 1D-CNN: (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [11]:
model = Sequential()

# 1D Convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Another Conv layer (optional)
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Flatten and Dense layers
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 32)            0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 6, 32)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 64)             6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2, 64)            0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 2, 64)             0

In [12]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 13s 3ms/step - loss: 0.0444 - accuracy: 0.9891 - val_loss: 0.0318 - val_accuracy: 0.9914
Epoch 2/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0303 - accuracy: 0.9921 - val_loss: 0.0276 - val_accuracy: 0.9914
Epoch 3/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0292 - accuracy: 0.9921 - val_loss: 0.0266 - val_accuracy: 0.9914
Epoch 4/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0274 - accuracy: 0.9922 - val_loss: 0.0253 - val_accuracy: 0.9914
Epoch 5/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0267 - accuracy: 0.9922 - val_loss: 0.0246 - val_accuracy: 0.9909
Epoch 6/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0264 - accuracy: 0.9924 - val_loss: 0.0237 - val_accuracy: 0.9916
Epoch 7/10
3307/3307 [==============================] - 12s 4ms/step - loss: 0.0257 - accuracy: 0.9924 - val_loss: 0.0237 - val_ac

In [13]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1772/1772 [==============================] - 3s 2ms/step - loss: 0.0218 - accuracy: 0.9932
Test Accuracy: 99.32%


In [14]:
score = model.evaluate(X_test,y_test)
print("ACCURACY : ",score)

1772/1772 [==============================] - 3s 2ms/step - loss: 0.0218 - accuracy: 0.9932
ACCURACY :  [0.021787749603390694, 0.9931558966636658]


In [15]:
score = model.evaluate(X_train,y_train)
print("ACCURACY : ",score)

4134/4134 [==============================] - 6s 1ms/step - loss: 0.0228 - accuracy: 0.9924
ACCURACY :  [0.022768428549170494, 0.9923795461654663]


In [16]:
num_features = data.shape[1] - 1  # Exclude the label column
print(f'Total number of features: {num_features}')

# Print all feature numbers
feature_numbers = list(range(1, num_features + 1))
print('Feature numbers:', feature_numbers)

Total number of features: 15
Feature numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [17]:
new_data = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]])  # 
new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data_scaled.reshape(new_data_scaled.shape[0], new_data_scaled.shape[1], 1)

# Predict
prediction = (model.predict(new_data_scaled) > 0.5).astype("int32")
print(f'Prediction: {prediction}')

1/1 [==============================] - 0s 211ms/step
Prediction: [[0]]


In [18]:
features = data.iloc[:, :-1]  # Adjust if label column is not the last column

# Print the feature names (if available) and their corresponding values
for i, column in enumerate(features.columns, start=1):
    print(f'Feature {i} ({column}):')
    print(features[column].values)
    print()

Feature 1 (timestamp):
[129.050634 129.051238 129.051434 ... 496.629047 496.629069 496.629091]

Feature 2 (processId):
[ 382  379    1 ... 7555 7555 7555]

Feature 3 (threadId):
[ 382  379    1 ... 7555 7555 7555]

Feature 4 (parentProcessId):
[   1    1    0 ... 7548 7548 7548]

Feature 5 (userId):
[ 101  100    0 ... 1001 1001 1001]

Feature 6 (mountNamespace):
[4026532232 4026532231 4026531840 ... 4026531840 4026531840 4026531840]

Feature 7 (processName):
[ 78  77  73 ... 233 233 233]

Feature 8 (hostName):
[0 0 0 ... 0 0 0]

Feature 9 (eventId):
[  41   41 1005 ...   42   42   42]

Feature 10 (eventName):
[40 40 32 ...  9  9  9]

Feature 11 (stackAddresses):
[1601 1158 2206 ... 4195 4195 4195]

Feature 12 (argsNum):
[3 3 4 ... 3 3 3]

Feature 13 (returnValue):
[  15   15    0 ... -114 -114 -114]

Feature 14 (args):
[ 6912  6912 12213 ... 25385 25407 25427]

Feature 15 (sus):
[0 0 0 ... 1 1 1]



In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

In [20]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for RNN: (samples, time steps, features)
# Here, we assume each sample is a single time step with multiple features
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [21]:
model = Sequential()

# Add a SimpleRNN layer
model.add(SimpleRNN(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Add a Dense output layer
model.add(Dense(units=1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100)               11600     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,701
Trainable params: 11,701
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 10s 3ms/step - loss: 0.0329 - accuracy: 0.9917 - val_loss: 0.0219 - val_accuracy: 0.9927
Epoch 2/10
3307/3307 [==============================] - 8s 2ms/step - loss: 0.0201 - accuracy: 0.9935 - val_loss: 0.0198 - val_accuracy: 0.9929
Epoch 3/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0182 - accuracy: 0.9936 - val_loss: 0.0183 - val_accuracy: 0.9930
Epoch 4/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0172 - accuracy: 0.9938 - val_loss: 0.0174 - val_accuracy: 0.9930
Epoch 5/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0163 - accuracy: 0.9938 - val_loss: 0.0169 - val_accuracy: 0.9934
Epoch 6/10
3307/3307 [==============================] - 6s 2ms/step - loss: 0.0154 - accuracy: 0.9938 - val_loss: 0.0168 - val_accuracy: 0.9933
Epoch 7/10
3307/3307 [==============================] - 7s 2ms/step - loss: 0.0147 - accuracy: 0.9940 - val_loss: 0.0158 - val_accuracy

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1772/1772 [==============================] - 2s 1ms/step - loss: 0.0117 - accuracy: 0.9948
Test Accuracy: 99.48%


In [24]:
new_data = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]])  # Replace with actual feature values
new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Predict
prediction = (model.predict(new_data_scaled) > 0.5).astype("int32")
print(f'Prediction: {prediction}')

1/1 [==============================] - 0s 186ms/step
Prediction: [[1]]


In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [26]:
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for LSTM: (samples, time steps, features)
# Here, we assume each sample is a single time step with multiple features
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [27]:
model = Sequential()

# Add an LSTM layer
model.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Add a Dense output layer
model.add(Dense(units=1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               46400     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 46,501
Trainable params: 46,501
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 12s 3ms/step - loss: 0.0384 - accuracy: 0.9909 - val_loss: 0.0227 - val_accuracy: 0.9927
Epoch 2/10
3307/3307 [==============================] - 8s 2ms/step - loss: 0.0203 - accuracy: 0.9936 - val_loss: 0.0203 - val_accuracy: 0.9928
Epoch 3/10
3307/3307 [==============================] - 9s 3ms/step - loss: 0.0184 - accuracy: 0.9936 - val_loss: 0.0188 - val_accuracy: 0.9930
Epoch 4/10
3307/3307 [==============================] - 11s 3ms/step - loss: 0.0174 - accuracy: 0.9936 - val_loss: 0.0181 - val_accuracy: 0.9929
Epoch 5/10
3307/3307 [==============================] - 12s 4ms/step - loss: 0.0165 - accuracy: 0.9937 - val_loss: 0.0176 - val_accuracy: 0.9929
Epoch 6/10
3307/3307 [==============================] - 13s 4ms/step - loss: 0.0157 - accuracy: 0.9936 - val_loss: 0.0176 - val_accuracy: 0.9930
Epoch 7/10
3307/3307 [==============================] - 13s 4ms/step - loss: 0.0152 - accuracy: 0.9938 - val_loss: 0.0165 - val_accu

In [29]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1772/1772 [==============================] - 6s 3ms/step - loss: 0.0115 - accuracy: 0.9950
Test Accuracy: 99.50%


In [30]:
new_data = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]) 
new_data_scaled = scaler.transform(new_data)
new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

# Predict
prediction = (model.predict(new_data_scaled) > 0.5).astype("int32")
print(f'Prediction: {prediction}')


1/1 [==============================] - 1s 809ms/step
Prediction: [[1]]


In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [32]:
# Assuming the last column is the label
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [33]:
# Initialize the model
model = Sequential()

# Add input layer and the first hidden layer
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))

# Add additional hidden layers
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
3307/3307 [==============================] - 14s 3ms/step - loss: 0.0256 - accuracy: 0.9927 - val_loss: 0.0185 - val_accuracy: 0.9930
Epoch 2/10
3307/3307 [==============================] - 7s 2ms/step - loss: 0.0182 - accuracy: 0.9936 - val_loss: 0.0163 - val_accuracy: 0.9930
Epoch 3/10
3307/3307 [==============================] - 5s 2ms/step - loss: 0.0152 - accuracy: 0.9937 - val_loss: 0.0154 - val_accuracy: 0.9934
Epoch 4/10
3307/3307 [==============================] - 6s 2ms/step - loss: 0.0123 - accuracy: 0.9949 - val_loss: 0.0114 - val_accuracy: 0.9950
Epoch 5/10
3307/3307 [==============================] - 5s 1ms/step - loss: 0.0105 - accuracy: 0.9956 - val_loss: 0.0135 - val_accuracy: 0.9954
Epoch 6/10
3307/3307 [==============================] - 6s 2ms/step - loss: 0.0089 - accuracy: 0.9965 - val_loss: 0.0086 - val_accuracy: 0.9966
Epoch 7/10
3307/3307 [==============================] - 6s 2ms/step - loss: 0.0076 - accuracy: 0.9971 - val_loss: 0.0099 - val_accuracy

In [35]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

1772/1772 [==============================] - 2s 1ms/step - loss: 0.0048 - accuracy: 0.9982
Test Accuracy: 99.82%


In [36]:
# Predict on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Classification report
print(classification_report(y_test, y_pred))

1772/1772 [==============================] - 2s 1ms/step
Accuracy: 0.9982184120936304
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      9044
           1       1.00      1.00      1.00     47647

    accuracy                           1.00     56691
   macro avg       1.00      1.00      1.00     56691
weighted avg       1.00      1.00      1.00     56691



In [37]:
score = model.evaluate(X_test,y_test)
print("ACCURACY : ",score)

1772/1772 [==============================] - 3s 2ms/step - loss: 0.0048 - accuracy: 0.9982
ACCURACY :  [0.004751901607960463, 0.9982184171676636]
